# 🖼️ Image Classification with CNNs & Transfer Learning

Complete deep learning pipeline for image classification.

## Topics
1. Building CNNs from scratch
2. Transfer learning with EfficientNet, ResNet
3. Data augmentation
4. Model evaluation and deployment

**Level**: Specialized  
**Time**: ~60 minutes

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 1. Data Loading with Augmentation

In [ ]:
# Define transforms
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Use CIFAR-10 for demo
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

classes = train_dataset.classes
print(f"Classes: {classes}")

## 2. Custom CNN Architecture

In [ ]:
class CustomCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.AdaptiveAvgPool2d(1)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 128), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

model_custom = CustomCNN(len(classes)).to(device)
print(f"Parameters: {sum(p.numel() for p in model_custom.parameters()):,}")

## 3. Transfer Learning with EfficientNet

In [ ]:
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

def create_efficientnet(num_classes, freeze_backbone=True):
    model = efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
    if freeze_backbone:
        for param in model.parameters():
            param.requires_grad = False
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    return model

model_effnet = create_efficientnet(len(classes)).to(device)
print(f"EfficientNet parameters: {sum(p.numel() for p in model_effnet.parameters() if p.requires_grad):,} trainable")

## 4. Training Loop

In [ ]:
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
    return total_loss / len(loader), correct / total

def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)
    return correct / total

# Train EfficientNet
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_effnet.classifier.parameters(), lr=0.001)

print("Training EfficientNet (transfer learning)...")
for epoch in range(3):
    loss, acc = train_epoch(model_effnet, train_loader, criterion, optimizer)
    val_acc = evaluate(model_effnet, test_loader)
    print(f"Epoch {epoch+1}: Loss={loss:.4f}, Train Acc={acc:.4f}, Val Acc={val_acc:.4f}")

## 5. Model Comparison

In [ ]:
from torchvision.models import resnet18, ResNet18_Weights, vgg16, VGG16_Weights

models_to_compare = {
    'Custom CNN': CustomCNN(len(classes)),
    'ResNet18': resnet18(weights=ResNet18_Weights.DEFAULT),
    'EfficientNet-B0': efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT),
}

# Modify classifiers
models_to_compare['ResNet18'].fc = nn.Linear(512, len(classes))
models_to_compare['EfficientNet-B0'].classifier[1] = nn.Linear(1280, len(classes))

comparison_results = []
for name, model in models_to_compare.items():
    params = sum(p.numel() for p in model.parameters())
    comparison_results.append({'Model': name, 'Parameters': f'{params:,}'})

import pandas as pd
print("\n📊 Model Comparison:")
display(pd.DataFrame(comparison_results))

## 6. Deployment

In [ ]:
# Save model
torch.save(model_effnet.state_dict(), 'efficientnet_classifier.pth')

# Export to ONNX for deployment
dummy_input = torch.randn(1, 3, 224, 224).to(device)
torch.onnx.export(model_effnet, dummy_input, 'model.onnx', input_names=['image'], output_names=['class'])
print("✅ Model exported to ONNX for deployment")

## 🎯 Key Takeaways
1. Transfer learning is faster than training from scratch
2. Data augmentation improves generalization
3. EfficientNet gives best accuracy/compute tradeoff
4. ONNX export enables cross-platform deployment